In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model, model_from_yaml
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
import tensorflow as tf
from keras import backend as K

tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
import pandas as pd

from skater.core.local_interpretation.dnni.deep_interpreter import DeepInterpreter
from skater.core.visualizer.text_relevance_visualizer import build_visual_explainer, show_in_notebook
from skater.util.dataops import convert_dataframe_to_dict

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Create a TensorFlow session and register it with Keras. It will use this session to initialize all the variables
sess = tf.Session()
K.set_session(sess)

In [3]:
# set parameters
max_features = 20000
maxlen = 80
batch_size = 32
embedding_dims = 128
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 3

### Load the Dataset
#### IMDB dataset: 
##### 1. http://ai.stanford.edu/~amaas//data/sentiment/
##### 2. http://ai.stanford.edu/~ang/papers/acl11-WordVectorsSentimentAnalysis.pdf ( Section 4.1 )

In [4]:
# The Dataset contains 50,000 reviews(Train:25,000 and Test:25,000)
# More info about the dataset: https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [5]:
# https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
# Reading raw text
INDEX_FROM = 3
# Get the {Word: Index} mapping
word_to_id = imdb.get_word_index()

def adjust_word_id_offset(word_id_dict):
    word_id_dict = {k:(v+INDEX_FROM) for k,v in word_id_dict.items()}
    word_id_dict["<PAD>"] = 0
    word_id_dict["<START>"] = 1
    word_id_dict["<UNK>"] = 2
    return word_id_dict

w_to_id = adjust_word_id_offset(word_to_id)

def get_raw_txt(word_id_dict, input_data):
    id_to_word = {value:key for key,value in word_id_dict.items()}
    return ' '.join([(id_to_word[_id] if _id in id_to_word else 'None') for _id in input_data])

r_t = get_raw_txt(w_to_id, x_train[20])
print(r_t + "\n")
print("Length: {}".format(len(r_t.split(' '))))

<START> shown in australia as <UNK> this incredibly bad movie is so bad that you become <UNK> and have to watch it to the end just to see if it could get any worse and it does the storyline is so predictable it seems written by a high school dramatics class the sets are pathetic but marginally better than the <UNK> and the acting is wooden br br the infant <UNK> seems to have been stolen from the props cupboard of <UNK> <UNK> there didn't seem to be a single original idea in the whole movie br br i found this movie to be so bad that i laughed most of the way through br br malcolm mcdowell should hang his head in shame he obviously needed the money

Length: 129


In [6]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [7]:
# Raw text post selecting the top most frequently occurring words
index_train = 20
r_t_r = get_raw_txt(w_to_id, x_train[index_train])
print(r_t_r + "\n")
print("Length: {}".format(len(r_t_r.split(' '))))

dramatics class the sets are pathetic but marginally better than the <UNK> and the acting is wooden br br the infant <UNK> seems to have been stolen from the props cupboard of <UNK> <UNK> there didn't seem to be a single original idea in the whole movie br br i found this movie to be so bad that i laughed most of the way through br br malcolm mcdowell should hang his head in shame he obviously needed the money

Length: 80


In [14]:
# Reference: https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(input_dim=max_features,
                    output_dim=embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

#### Summarize the Model

In [ ]:
model.summary()

### Persist the model for future use

In [ ]:
# Save and persist the trained keras model in YAML format
model_yaml = model.to_yaml()
with open("model_cnn_imdb_{}.yaml".format(epochs), "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model_cnn_imdb_{}.h5".format(epochs))
print("Save model to disk")

### Load the saved model

In [8]:
# load the model
# Learning phase is set to '0' as we are not training
K.set_learning_phase(0)
yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
print("Load model from disk")


# Validate model performance with the reload of persisted model
loaded_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
score, acc = loaded_model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print("\n\n")
print('Test score:', score)
print('Test accuracy:', acc)

Load model from disk
24800/25000 [============================>.] - ETA: 0s


Test score: 0.5148597713279724
Test accuracy: 0.8376


### Lets ask Skater to help us in interpreting the model

In [9]:
index = 1
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = np.array([x_test[index]])
    ys = np.array([y_test[index]])

    print('Predicted class : {}'.format(loaded_model.predict_classes(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs});
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('integ_grad', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out);

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-04-23 06:46:36,842 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class : [[1]]
Ground Truth: [1]


In [10]:
# building a dataframe with columns 'features' and 'relevance scores'
# Since, the relevance score is compute over the embedding vector, we aggregate it by computing 'mean'
# over the embedding to get scalar coefficient for the features
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
relevance_scores_df.describe()

count    80.000000
mean      0.000082
std       0.007579
min      -0.032703
25%      -0.001871
50%       0.000000
75%       0.002395
max       0.020106
dtype: float64

In [11]:
# # Retrieve the text
r_t_test = get_raw_txt(w_to_id, x_test[index])
print(r_t_test)

as he spouts the one liners out i also like the scenes with <UNK> at the beginning find her very sexy when she's wearing all that fetish gear i can't be the only one surely i personally think bride of chucky is a fantastic film total entertainment from start to finish great humour horror in equal measure at only 85 minutes long it never becomes boring or dull a personal favourite of mine watch it as soon as you can


#### Visualize the results

In [12]:
#words_scores_dict = convert_dataframe_to_dict('features', 'relevance_scores', words_scores_df)
build_visual_explainer(r_t_test, relevance_scores_df, highlight_oov=True, enable_plot=True)

2018-04-23 06:46:39,860 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as feature_relevance.png
2018-04-23 06:46:39,861 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: feature_relevance.png
2018-04-23 06:46:39,884 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered


In [13]:
show_in_notebook('./rendered.html')

### Generating conditional adversarial examples

#### Generating text by deleting or updating textual information

In [14]:
new_txt = r_t_test.replace("favourite", "<UNK>")
print(new_txt)

as he spouts the one liners out i also like the scenes with <UNK> at the beginning find her very sexy when she's wearing all that fetish gear i can't be the only one surely i personally think bride of chucky is a fantastic film total entertainment from start to finish great humour horror in equal measure at only 85 minutes long it never becomes boring or dull a personal <UNK> of mine watch it as soon as you can


In [15]:
# Convert the dataset to engineered feature format
# Reference: https://stackoverflow.com/questions/42964375/how-to-input-new-text-for-prediction-in-keras-while-using-an-inbuilt-dataset

def input_formatter_imdb(input_txt, word_index_mapping):
    
    x_i_test = [[word_index_mapping[wrds] if wrds in word_index_mapping.keys() else word_index_mapping["<UNK>"] 
                 for wrds in input_txt.split(' ')]]
    x_i_test = sequence.pad_sequences(x_i_test, maxlen=maxlen)
    txt_vector = np.array([x_i_test.flatten()])
    return txt_vector

In [16]:
input_vector = input_formatter_imdb(new_txt, w_to_id)
input_vector

array([[   17,    29, 14578,     4,    31,  2433,    46,    13,    82,
           40,     4,   139,    19,     2,    33,     4,   454,   169,
           41,    55,  1279,    54,   442,  1658,    32,    15,  7717,
         5745,    13,   191,    30,     4,    64,    31,  1348,    13,
         1276,   104,  3452,     7, 16495,     9,     6,   777,    22,
          964,   722,    39,   380,     8,  1363,    87,  1285,   189,
           11,  3215,  4160,    33,    64,  7304,   234,   196,    12,
          115,   461,   357,    42,   753,     6,   965,     2,     7,
         1923,   106,    12,    17,   515,    17,    25,    70]],
      dtype=int32)

In [17]:
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = input_vector
    ys = np.array([1])

    print('Predicted class : {}'.format(loaded_model.predict_classes(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs})
    print(embedding_out.shape)
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('integ_grad', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out);

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-04-23 06:46:48,970 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class : [[1]]
Ground Truth: [1]
(1, 80, 128)


In [23]:
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
relevance_scores_df.describe()

# Retrieve the text
_in = input_vector.reshape(-1)
r_t = get_raw_txt(w_to_id, _in)
print("New Document:\n\n {}".format(r_t))

build_visual_explainer(r_t, relevance_scores_df, highlight_oov=True, enable_plot=True)
show_in_notebook('./rendered.html')

New Document:

 as he spouts the one liners out i also like the scenes with <UNK> at the beginning find her very sexy when she's wearing all that fetish gear i can't be the only one surely i personally think bride of chucky is a fantastic film total entertainment from start to finish great humour horror in equal measure at only 85 minutes long it never becomes boring or dull a personal <UNK> of mine watch it as soon as you can


2018-04-23 06:48:23,702 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as feature_relevance.png
2018-04-23 06:48:23,703 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: feature_relevance.png
2018-04-23 06:48:23,724 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered


In [19]:
# replaciing words in a sentence
import re
words_to_replace = ["great", 'is', 'it', 'finish', 'out', 'fantastic', 'entertainment', 'very']
regex_builder = re.compile('|'.join(map(re.escape, words_to_replace)))
new_txt2 = regex_builder.sub("<UNK>" , new_txt)
print(new_txt2)

as he sp<UNK>s the one liners <UNK> i also like the scenes w<UNK>h <UNK> at the beginning find her <UNK> sexy when she's wearing all that fet<UNK>h gear i can't be the only one surely i personally think bride of chucky <UNK> a <UNK> film total <UNK> from start to <UNK> <UNK> humour horror in equal measure at only 85 minutes long <UNK> never becomes boring or dull a personal <UNK> of mine watch <UNK> as soon as you can


In [20]:
input_vector2 = input_formatter_imdb(new_txt2, w_to_id)

In [21]:
K.set_learning_phase(0)
with DeepInterpreter(session=K.get_session()) as di:
    print("learning_phase {}".format(K.learning_phase()))
    yaml_file = open('model_cnn_imdb_{}.yaml'.format(epochs), 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    
    loaded_model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    loaded_model.load_weights('model_cnn_imdb_{}.h5'.format(epochs))
    print("Load model from disk")    
    
    # Input data
    xs = input_vector2
    ys = np.array([1])

    print('Predicted class : {}'.format(loaded_model.predict_classes(np.array([x_test[index]]))))
    print('Ground Truth: {}'.format(ys))
    
    embedding_tensor = loaded_model.layers[0].output
    input_tensor = loaded_model.layers[0].input
    
    embedding_out = di.session.run(embedding_tensor, {input_tensor: xs})
    print(embedding_out.shape)
    # Using Integrated Gradient for computing feature relevance
    relevance_scores = di.explain('integ_grad', loaded_model.layers[-2].output * ys, 
                                  loaded_model.layers[1].input, embedding_out);

learning_phase 0
Load model from disk
1/1 [==============================] - 0s

2018-04-23 06:47:05,518 - IntegratedGradients - INFO - Executing operations to compute relevance using Integrated Gradient



Predicted class : [[1]]
Ground Truth: [1]
(1, 80, 128)


In [24]:
relevance_scores_df = pd.DataFrame(relevance_scores[0]).mean(axis=1)
relevance_scores_df.describe()

# Retrieve the text
_in = input_vector2.reshape(-1)
r_t = get_raw_txt(w_to_id, _in)
print("Original Document:\n\n {}".format(r_t))

# words_ = r_t.split(' ')

# # merging the dataframe columnwise
# words_df = pd.DataFrame({'features': words_})
# scores_df = pd.DataFrame({'relevance_scores': relevance_scores_df.tolist()})
# words_scores_df = words_df.join(scores_df)
# words_scores_df.describe()

# words_scores_dict = convert_dataframe_to_dict('features', 'relevance_scores', words_scores_df)

k_args = {'plot_name' : 'plt3.png'}
build_visual_explainer(r_t, relevance_scores_df, highlight_oov=True, enable_plot=True, **k_args)
show_in_notebook('./rendered.html')

Original Document:

 as he <UNK> the one liners <UNK> i also like the scenes <UNK> <UNK> at the beginning find her <UNK> sexy when she's wearing all that <UNK> gear i can't be the only one surely i personally think bride of chucky <UNK> a <UNK> film total <UNK> from start to <UNK> <UNK> humour horror in equal measure at only 85 minutes long <UNK> never becomes boring or dull a personal <UNK> of mine watch <UNK> as soon as you can


2018-04-23 06:48:37,781 - skater.core.visualizer.text_relevance_visualizer - INFO - Rank order feature relevance based on input created and saved as plt3.png
2018-04-23 06:48:37,782 - skater.core.visualizer.text_relevance_visualizer - INFO - Relevance plot name: plt3.png
2018-04-23 06:48:37,801 - skater.core.visualizer.text_relevance_visualizer - INFO - Visual Explainer built, use show_in_notebook to render in Jupyter style Notebooks: rendered
